In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/3.4/data/haarcascades/haarcascade_frontalface_default.xml

--2022-09-13 15:58:31--  https://raw.githubusercontent.com/opencv/opencv/3.4/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml.1’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.03s   

2022-09-13 15:58:31 (29.9 MB/s) - ‘haarcascade_frontalface_default.xml.1’ saved [930127/930127]



In [ ]:
import cv2
import jpeg4py
import time
import PIL
import io
import numpy as np
from matplotlib import pyplot as pyplot
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

In [19]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  return binary

In [ ]:
from os import O_NONBLOCK
kcf_tracker = None
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
  data = take_photo()
  res = PIL.Image.open(io.BytesIO(data))
  pix = np.array(res)
  pix = cv2.cvtColor(pix, cv2.COLOR_RGB2BGR)

  kcf_tracker_result = None;

  if kcf_tracker is not None:
    ok, box = kcf_tracker.update(pix)
    if ok:
      kcf_tracker_result = box;
    
  gray = cv2.cvtColor(pix, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.3, 3)
  print(len(faces))


  if len(faces) != 0 and kcf_tracker is None:
    kcf_tracker = cv2.TrackerKCF_create()
    kcf_tracker.init(pix, faces[0])


  for (x, y, w, h) in faces:
    cv2.rectangle(pix, (x, y), (x + w, y + h), (255, 0, 0), 2)
    cv2.putText(pix, 'Detector', (x, y - 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 1,
                  (255, 0, 0), 2, cv2.LINE_AA)
      
  if kcf_tracker_result is not None:
    (x, y, w, h) = map(int, kcf_tracker_result)
    cv2.rectangle(pix, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.putText(pix, 'Tracker', (x, y - 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 1,
                  (0, 0, 255), 2, cv2.LINE_AA)
      
  cv2_imshow(pix)
  time.sleep(2)
